In [1]:
from bayes_opt import BayesianOptimization
import numpy as np
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from matplotlib import gridspec

from trainlib.ConfigFileHandler import ConfigFileHandler

In [2]:
def load_file(path, keys):
    confhandler = ConfigFileHandler()
    confhandler.load_configuration(path)
    
    retval = {}
    
    for section_name in confhandler.get_sections():
        cur_section = confhandler.get_section(section_name)
        
        for key in keys:
            if not key in retval:
                retval[key] = []
                
            retval[key].append(float(cur_section[key]))
            
    return retval

In [3]:
def posterior(bo, x):
    bo.gp.fit(bo.X, bo.Y)
    mu, sigma = bo.gp.predict(x, return_std=True)
    return mu, sigma

def plot_gp(bo, x):
    
    fig = plt.figure(figsize=(16, 10))
    axis = plt.subplot(111)
    
    mu, sigma = posterior(bo, x)
    axis.plot(bo.X.flatten(), bo.Y, 's', markersize=8, label=u'observations', color='black')
    axis.plot(x, mu, '--', color='black', label='gaussian process mean')

    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - 1.9600 * sigma, (mu + 1.9600 * sigma)[::-1]]),
        alpha=.6, fc='lightblue', ec='None', label='95% confidence interval')
    
    axis.set_xlim((min(x), max(x)))
    axis.set_ylim((None, None))
    axis.set_ylabel('utility', fontdict={'size':20})
    axis.set_xlabel('ggH prior', fontdict={'size':20})
    
    axis.legend(loc = 3)
    
    return fig

In [4]:
init_dict = load_file("/data_CMS/cms/wind/180424_sequential_prior_optimization/number_layers_2.0_number_neurons_64.0/priors_tree/evaluations_ggH.txt", ["target", "ggh_prior"])

attempting to load configuration file from /data_CMS/cms/wind/180424_sequential_prior_optimization/number_layers_2.0_number_neurons_64.0/priors_tree/evaluations_ggH.txt


In [5]:
gp_params = {'kernel': 1.0 * Matern(length_scale = 0.01, length_scale_bounds = (1e-5, 1e5), nu = 1.5),
             'alpha':1e-4}

In [8]:
target = lambda x: x

In [9]:
x = np.linspace(1.0, 1.7, 1000).reshape(-1,1)

In [10]:
bo = BayesianOptimization(target, {'ggh_prior': (1.0, 1.7)})

In [11]:
bo.initialize(init_dict)

In [13]:
bo.maximize(init_points=0, n_iter=0, acq='poi', kappa=5, xi = 0.1, **gp_params)

Initialization
-------------------------------------------
 Step |   Time |      Value |   ggh_prior | 
    1 | 00m00s |    0.00894 |      1.5703 | 
    2 | 00m00s |    0.01548 |      1.5220 | 
    3 | 00m00s |    0.00019 |      1.0000 | 
    4 | 00m00s |    0.00309 |      1.2289 | 
    5 | 00m00s |    0.01872 |      1.5029 | 
    6 | 00m00s |   -0.05792 |      1.7000 | 
    7 | 00m00s |    0.01874 |      1.3742 | 
    8 | 00m00s |    0.00860 |      1.1148 | 
    9 | 00m00s |    0.00784 |      1.3055 | 
   10 | 00m00s |    0.01794 |      1.4366 | 
   11 | 00m00s |    0.00283 |      1.1698 | 
   12 | 00m00s |    0.00371 |      1.0580 | 
   13 | 00m00s |    0.01523 |      1.4090 | 
   14 | 00m00s |    0.00353 |      1.0000 | 
   15 | 00m00s |    0.01893 |      1.3345 | 
   16 | 00m00s |    0.01828 |      1.4663 | 
   17 | 00m00s |   -0.01413 |      1.6067 | 
   18 | 00m00s |   -0.00070 |      1.2655 | 
   19 | 00m00s |    0.01718 |      1.3865 | 
   20 | 00m00s |    0.01493 |      1.0999

In [14]:
fig = plot_gp(bo, x)

In [15]:
plt.savefig("gaussian_process.pdf")